# Práctica 1 - Self-Organising Maps
## Preparación de entorno
#### Importar librerías de código

In [ ]:
from matplotlib import patches as patches
import numpy as  np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

#### Dataset que se va a utilizar para el entrenamiento

In [ ]:
# Código para obtener el Dataset que se va a usar en el entrenamiento

def create_color_dataset(num_samples):
    colors = np.random.randint(0, 256, size=(num_samples, 3))
    return colors

num_samples = 100
datos = create_color_dataset(num_samples)

print("Dataset shape:", datos.shape)


## SOM Setup


In [ ]:
# Inicializa parámetros del SOM
lado_mapa = 100    # Tamaño del mapa
periodo = 100      # Número de iteraciones para el entrenamiento
learning_rate = 0.1 # Tasa de aprendizaje inicial
normalizar_datos = True  # Normalizar los datos RGB al rango [0,1]

In [ ]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.
num_entradas = datos.shape[1]
num_datos = datos.shape[0] 

# Calcula el vecindario inicial
vecindario = lado_mapa // 2

# Normaliza los datos si es necesario
if normalizar_datos:
    datos = datos / 255.0
    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
#matriz_pesos = np.random.random((dimensiones de la matriz de pesos))

matriz_pesos = np.random.random((lado_mapa, lado_mapa, num_entradas))
if not normalizar_datos:
    matriz_pesos = matriz_pesos * 255

#### Funciones para entrenar/clasificar

In [ ]:
def calcular_bmu(patron_de_entrada, matriz_de_pesos):
    """
    Encuentra la BMU para un patrón de entrada.
    Entradas: (patrón_de_entrada, matriz_de_pesos)
    Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganador
               bmu_idx: coordenadas de la neurona ganadora 
    """
    # Cálculo de distancias euclídeas
    distancias = np.linalg.norm(matriz_de_pesos - patron_de_entrada, axis=2)
    
    # Encontrar índice de la BMU
    bmu_idx = np.unravel_index(np.argmin(distancias), distancias.shape)
    
    # Devolver bmu como vector de pesos y bmu_idx como coordenadas
    return matriz_de_pesos[bmu_idx], bmu_idx

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
   return lr_inicial * (1 - i / n_iteraciones)

In [ ]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
   return vecindario_inicial - (vecindario_inicial - 1) * (i / n_iteraciones)

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
   return np.exp(-distancia_BMU**2 / (2* (vecindario_actual**2)))

#### Funciones para dibujar la salida de la red

In [ ]:
def pintar_mapa(matriz_valores, iteracion=None, es_inicial=False):
    fig = plt.figure()
    
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_valores.shape[0]+1))
    ax.set_ylim((0, matriz_valores.shape[1]+1))
    
    # Choose the title based on parameters
    if es_inicial:
        ax.set_title('Matriz de Pesos Inicial')
    elif iteracion is not None:
        ax.set_title(f'SOM después de {iteracion} Iteraciones')
    else:
        ax.set_title(f'SOM después de {periodo} Iteraciones')

    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1,
                         facecolor=matriz_valores[x-1,y-1,:],
                         edgecolor='none'))
    plt.show()

## SOM Entrenamiento

In [ ]:
def entrenar_som(datos, lado_mapa, periodo, learning_rate, normalizar_datos=True):
    # Inicializa la matriz de pesos
    num_entradas = datos.shape[1]
    matriz_pesos = np.random.random((lado_mapa, lado_mapa, num_entradas))
    
    # Visualiza la matriz de pesos inicial
    matriz_pesos_inicial = matriz_pesos.copy()
    if not normalizar_datos:
        matriz_pesos_inicial = matriz_pesos_inicial * 255
    pintar_mapa(matriz_pesos_inicial, es_inicial=True)
    
    # Entrenamiento
    num_datos = datos.shape[0]
    for i in range(periodo):
        for idx in range(num_datos):
            patron = datos[idx]
            
            # Encuentra la BMU
            bmu, bmu_idx = calcular_bmu(patron, matriz_pesos)
            
            # Calcula parámetros actuales
            lr_actual = variacion_learning_rate(learning_rate, i, periodo)
            v_actual = variacion_vecindario(vecindario, i, periodo)
            
            # Calcula todas las distancias a la BMU
            x_coords = np.arange(matriz_pesos.shape[0])[:, None]
            y_coords = np.arange(matriz_pesos.shape[1])[None, :]
            dist_bmu = np.sqrt((x_coords - bmu_idx[0])**2 + (y_coords - bmu_idx[1])**2)

            # Inicializa matriz de influencia
            influencia = np.zeros_like(dist_bmu)
            
            # Función discreta: 1 para adyacentes (distancia = 1)
            influencia[dist_bmu == 1] = 1
            
            # Dentro del radio: usa función de decay
            mascara_radio = dist_bmu <= v_actual
            influencia[mascara_radio] = decay(dist_bmu[mascara_radio], v_actual)
            
            # Actualiza todos los pesos
            delta = lr_actual * influencia[:, :, None] * (patron - matriz_pesos)
            matriz_pesos += delta
            
        # Muestra progreso
        if (i + 1) % 100 == 0:
            print(f'Iteración {i + 1}/{periodo}')
            matriz_pesos_para_plot = matriz_pesos.copy()
            if not normalizar_datos:
                matriz_pesos_para_plot = matriz_pesos_para_plot * 255
            pintar_mapa(matriz_pesos_para_plot, iteracion=i + 1)
    
    return matriz_pesos

# Estos son los mejores hiperparámetros
matriz_pesos_final = entrenar_som(
        datos, 
        lado_mapa=50, 
        periodo=100, 
        learning_rate=0.1
    )

In [ ]:
def calcular_matriz_activaciones(datos, matriz_pesos):
    # Inicializar matriz de activaciones
    lado_mapa = matriz_pesos.shape[0]
    matriz_activaciones = np.zeros((lado_mapa, lado_mapa))
    
    # Contar activaciones para cada patrón
    for patron in datos:
        # Usar la misma función calcular_bmu que usamos en el entrenamiento
        _, bmu_idx = calcular_bmu(patron, matriz_pesos)
        matriz_activaciones[bmu_idx[0], bmu_idx[1]] += 1
        
    return matriz_activaciones

def visualizar_mapas_som(matriz_pesos, matriz_activaciones):
    # 1. Mapa de neuronas activadas con cuadrados y texto
    fig1, ax1 = plt.subplots(figsize=(8, 8))
    ax1.set_facecolor('black')  # Fondo 
    fig1.patch.set_facecolor('white')  # Fondo de la figura 
    
    for i in range(matriz_pesos.shape[0]):
        for j in range(matriz_pesos.shape[1]):
            if matriz_activaciones[i,j] > 0:
                ax1.add_patch(plt.Rectangle((i-0.5, j-0.5), 1, 1, 
                                           facecolor=matriz_pesos[i,j],
                                           edgecolor='black',  # Bordes negros
                                           linewidth=1))
        
    ax1.set_title('Mapa de Neuronas Activadas', color='black', pad=20)
    ax1.set_xlim(-1, matriz_pesos.shape[0])
    ax1.set_ylim(-1, matriz_pesos.shape[1])
    ax1.set_xticks([])  # Sin ticks en los ejes
    ax1.set_yticks([])
    ax1.grid(False)  # Sin grid
    plt.show()
    
    # 2. Mapa de clasificación 3D con columnas coloreadas por activación
    fig2 = plt.figure(figsize=(9, 8)) 
    ax2 = fig2.add_subplot(111, projection='3d')
    x, y = np.meshgrid(range(matriz_activaciones.shape[0]), 
                       range(matriz_activaciones.shape[1]))
    z = matriz_activaciones.T
    
    dx = dy = 0.8
    norm = plt.Normalize(vmin=0, vmax=matriz_activaciones.max())
    cmap = cm.viridis
    
    # Ajustar escala Z para intervalos de 1
    ax2.set_zticks(np.arange(0, matriz_activaciones.max() + 1, 1))
    
    for i in range(matriz_activaciones.shape[0]):
        for j in range(matriz_activaciones.shape[1]):
            if matriz_activaciones[i,j] > 0:
                color = cmap(norm(matriz_activaciones[i,j]))
                ax2.bar3d(i-dx/2, j-dy/2, 0, dx, dy, z[j,i], 
                         color=color, shade=True, edgecolor='blue', linewidth=0.5)
    
    ax2.view_init(elev=25, azim=45)  # Lower elevation angle
    ax2.set_title('Mapa de Clasificación 3D', color='black', pad=20)
    ax2.set_box_aspect((1, 1, 1.1))  # Reduced height aspect ratio
    
    # Alinear y ajustar barra de color
    mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
    cbar = plt.colorbar(mappable, ax=ax2, 
                       ticks=np.arange(0, matriz_activaciones.max() + 1, 1),
                       pad=0.1,
                       shrink=0.7,     # Make colorbar shorter
                       aspect=20)       # Make colorbar wider
    cbar.set_label('Activaciones', color='black')
    cbar.ax.yaxis.set_tick_params(color='black')
    cbar.ax.tick_params(colors='black')
    fig2.patch.set_facecolor('white')
    ax2.grid(True)
    plt.tight_layout()
    plt.show()
    
    # 3. Mapa de distancias normalizado
    fig3, ax3 = plt.subplots(figsize=(8, 8))
    fig3.patch.set_facecolor('white')  # Fondo blanco
    ax3.set_facecolor('white')  # Fondo del eje blanco
    
    # Calcular distancias
    distancias = np.zeros_like(matriz_activaciones, dtype=float)
    for i in range(matriz_pesos.shape[0]):
        for j in range(matriz_pesos.shape[1]):
            vecinos = []
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    if di == 0 and dj == 0:
                        continue
                    ni, nj = i + di, j + dj
                    if 0 <= ni < matriz_pesos.shape[0] and 0 <= nj < matriz_pesos.shape[1]:
                        dist = np.linalg.norm(matriz_pesos[i,j] - matriz_pesos[ni,nj])
                        vecinos.append(dist)
            distancias[i,j] = np.mean(vecinos) if vecinos else 0
    
    # Normalizar distancias
    distancias_norm = distancias / distancias.max() if distancias.max() > 0 else distancias
    
    # Modificar visualización del heatmap con barra más pequeña
    sns.heatmap(distancias_norm, ax=ax3,
                cmap='viridis',
                cbar_kws={
                    'label': 'Distancia Normalizada',
                    'shrink': 0.7,  # Make colorbar 70% of original size
                    'aspect': 20,   # Make colorbar thinner
                    'pad': 0.02     # Adjust padding
                },
                square=True,
                xticklabels=False,
                yticklabels=False,
                linewidths=0.0)
    
    ax3.set_title('Mapa de Distancias', color='black', pad=20)
    ax3.collections[0].colorbar.ax.yaxis.set_tick_params(color='black')
    ax3.collections[0].colorbar.ax.tick_params(colors='black')
    ax3.collections[0].colorbar.set_label('Distancia Normalizada', color='black')
    plt.tight_layout()
    plt.show()

# Generar visualizaciones
matriz_activaciones = calcular_matriz_activaciones(datos, matriz_pesos_final)
visualizar_mapas_som(matriz_pesos_final, matriz_activaciones)

## SOM Clasificación

In [ ]:
def calcular_error_cuantificacion(datos, matriz_pesos):
    """Calcula el error de cuantificación medio"""
    distancias = []
    for patron in datos:
        bmu, _ = calcular_bmu(patron, matriz_pesos)
        distancia = np.linalg.norm(patron - bmu)
        distancias.append(distancia)
    return np.mean(distancias)

def calcular_error_topologico(datos, matriz_pesos, lado_mapa):
    """Calcula el error topológico"""
    errores = 0
    for patron in datos:
        # Encontrar primera BMU
        _, bmu1_idx = calcular_bmu(patron, matriz_pesos)
        
        # Encontrar segunda BMU
        distancias = np.linalg.norm(matriz_pesos - patron, axis=2)
        distancias[bmu1_idx[0], bmu1_idx[1]] = np.inf
        bmu2_idx = np.unravel_index(np.argmin(distancias), distancias.shape)
        
        # Verificar si son adyacentes
        dx = abs(bmu1_idx[0] - bmu2_idx[0])
        dy = abs(bmu1_idx[1] - bmu2_idx[1])
        if dx > 1 or dy > 1:
            errores += 1
    return errores / len(datos)

def contar_clases_activadas(datos, matriz_pesos):
    """Cuenta el número de neuronas activadas"""
    bmus = set()
    for patron in datos:
        _, bmu_idx = calcular_bmu(patron, matriz_pesos)
        bmus.add(bmu_idx)
    return len(bmus)

configs = [
    {"lado_mapa": 10, "periodo": 500, "learning_rate": 0.1},
    {"lado_mapa": 20, "periodo": 500, "learning_rate": 0.1},
    {"lado_mapa": 30, "periodo": 500, "learning_rate": 0.1},
    {"lado_mapa": 50, "periodo": 500, "learning_rate": 0.1},
    {"lado_mapa": 75, "periodo": 500, "learning_rate": 0.1},
    {"lado_mapa": 100, "periodo": 500, "learning_rate": 0.1}
]

resultados = []

datos = create_color_dataset(100)
datos = datos / 255.0 

In [ ]:
for config in configs:
    print(f"\nEntrenando SOM de Lado: {config['lado_mapa']}")
    
    # Initialize vecindario for this configuration
    global vecindario
    vecindario = config["lado_mapa"] // 2
    
    # Train SOM with current configuration
    matriz_pesos = entrenar_som(
        datos=datos,
        lado_mapa=config["lado_mapa"],
        periodo=config["periodo"],
        learning_rate=config["learning_rate"],
        normalizar_datos=True  # Data is already normalized
    )
    
    # Calculate metrics
    eq_error = calcular_error_cuantificacion(datos, matriz_pesos)
    top_error = calcular_error_topologico(datos, matriz_pesos, config["lado_mapa"])
    n_clases = contar_clases_activadas(datos, matriz_pesos)
    
    resultados.append({
        "Lado mapa": config["lado_mapa"],
        "Nº Clases": n_clases,
        "Error Topologico": round(top_error, 4),
        "Error de Cuantificación": round(eq_error, 4)
    })

# Crear y mostrar DataFrame con resultados
df_resultados = pd.DataFrame(resultados)
print("\nResultados de los experimentos:")
print(df_resultados.to_string(index=False))

# Guardar resultados
df_resultados.to_csv('resultados_som.csv', index=False)

In [ ]:
# Crear tabla con los resultados obtenidos
df = pd.DataFrame(resultados)
print("\nTabla de resultados:")
print(df.to_string(index=False))

# Guardar la tabla en formato CSV
df.to_csv('resultados_som.csv', index=False)

# Configurar estilo de gráficos
plt.style.use('seaborn')

# Crear figura con 3 subplots
fig, axes = plt.subplots(3, 1, figsize=(12, 15))

# Gráfica de error de cuantificación
axes[0].plot(df["Lado mapa"], df["Error de Cuantificación"], marker='o', color='red', linestyle='-')
axes[0].set_title("Error de Cuantificación vs Tamaño del Mapa (LR=0.01, Periodo=500)")
axes[0].set_xlabel("Lado del Mapa")
axes[0].set_ylabel("Error de Cuantificación")
axes[0].grid(True)

# Gráfica de error topológico
axes[1].plot(df["Lado mapa"], df["Error Topologico"], marker='o', color='red', linestyle='-')
axes[1].set_title("Error Topológico vs Tamaño del Mapa (LR=0.01, Periodo=500)")
axes[1].set_xlabel("Lado del Mapa")
axes[1].set_ylabel("Error Topológico")
axes[1].grid(True) 

# Gráfica de número de clases
axes[2].plot(df["Lado mapa"], df["Nº Clases"], marker='o', color='red', linestyle='-')
axes[2].set_title("Número de Clases Activadas vs Tamaño del Mapa")
axes[2].set_xlabel("Lado del Mapa")
axes[2].set_ylabel("Número de Clases")
axes[2].grid(True)

# Ajustar espaciado
plt.tight_layout()
plt.show()

## SOM Prueba

In [ ]:
matriz_pesos_final = entrenar_som(
        datos, 
        lado_mapa=100, 
        periodo=500, 
        learning_rate=0.1
    )

In [ ]:
def clasificar_colores_prueba():
    """Clasifica el conjunto de colores de prueba usando el SOM entrenado"""
    # Colores de prueba
    colores_prueba = np.array([
        [255, 255, 255],  # Blanco
        [255, 0, 0],      # Rojo
        [0, 255, 0],      # Verde
        [0, 0, 255],      # Azul
        [255, 255, 0],    # Amarillo
        [255, 0, 255],    # Magenta
        [0, 255, 255],    # Cian
        [0, 0, 0]         # Negro
    ]) / 255.0  # Normalizar colores

    # Clasificar cada color y calcular métricas
    resultados = []
    matriz_activaciones = np.zeros((lado_mapa, lado_mapa))
    
    for i, color in enumerate(colores_prueba):
        _, bmu_idx = calcular_bmu(color, matriz_pesos_final)
        color_asignado = matriz_pesos_final[bmu_idx[0], bmu_idx[1]]
        matriz_activaciones[bmu_idx[0], bmu_idx[1]] += 1
        
        resultados.append({
            'Color Original': color * 255,
            'BMU': bmu_idx,
            'Color Asignado': color_asignado * 255,
            'Error': np.linalg.norm(color - matriz_pesos_final[bmu_idx[0], bmu_idx[1]])
        })
    
    # Calcular métricas globales
    eq_error = calcular_error_cuantificacion(colores_prueba, matriz_pesos_final)
    top_error = calcular_error_topologico(colores_prueba, matriz_pesos_final, lado_mapa)
    n_clases = contar_clases_activadas(colores_prueba, matriz_pesos_final)
    
    # Mostrar resultados individuales
    for i, res in enumerate(resultados):
        print(f"\nColor {i+1}:")
        print(f"Original RGB: {res['Color Original']}")
        print(f"BMU: {res['BMU']}")
        print(f"Asignado RGB: {res['Color Asignado']}")
        print(f"Error: {res['Error']:.4f}")
    
    # Mostrar métricas globales
    print("\nMétricas Globales:")
    print(f"Error de Cuantificación: {eq_error:.4f}")
    print(f"Error Topológico: {top_error:.4f}")
    print(f"Número de Clases Activadas: {n_clases}")
    
    return resultados, matriz_activaciones

# Ejecutar clasificación y visualización
resultados, matriz_activaciones = clasificar_colores_prueba()

In [ ]:
def calcular_matriz_activaciones(datos, matriz_pesos):
    """Calcula la matriz de activaciones del SOM"""
    lado_mapa = matriz_pesos.shape[0]
    matriz_activaciones = np.zeros((lado_mapa, lado_mapa))
    
    for patron in datos:
        _, bmu_idx = calcular_bmu(patron, matriz_pesos)
        matriz_activaciones[bmu_idx[0], bmu_idx[1]] += 1
        
    return matriz_activaciones

def clasificar_colores_prueba():
    """Clasifica el conjunto de colores de prueba usando el SOM entrenado"""
    # Colores de prueba
    colores_prueba = np.array([
        [255, 255, 255],  # Blanco
        [255, 0, 0],      # Rojo
        [0, 255, 0],      # Verde
        [0, 0, 255],      # Azul
        [255, 255, 0],    # Amarillo
        [255, 0, 255],    # Magenta
        [0, 255, 255],    # Cian
        [0, 0, 0]         # Negro
    ]) / 255.0  # Normalizar colores
    
    # Clasificar cada color
    resultados = []
    matriz_activaciones = np.zeros((lado_mapa, lado_mapa))
    
    for i, color in enumerate(colores_prueba):
        _, bmu_idx = calcular_bmu(color, matriz_pesos_final)
        color_asignado = matriz_pesos_final[bmu_idx[0], bmu_idx[1]]
        matriz_activaciones[bmu_idx[0], bmu_idx[1]] += 1
        
        resultados.append({
            'Color Original': color * 255,
            'BMU': bmu_idx,
            'Color Asignado': color_asignado * 255,
            'Error': np.linalg.norm(color - matriz_pesos_final[bmu_idx[0], bmu_idx[1]])
        })
    
    # Mostrar resultados individuales
    for i, res in enumerate(resultados):
        print(f"\nColor {i+1}:")
        print(f"Original RGB: {res['Color Original']}")
        print(f"BMU: {res['BMU']}")
        print(f"Asignado RGB: {res['Color Asignado']}")
        print(f"Error: {res['Error']:.4f}")
    
    return resultados, matriz_activaciones

def visualizar_mapas_som(matriz_pesos, matriz_activaciones):
    """Visualiza los mapas del SOM"""
    # 1. Mapa de neuronas activadas con cuadrados
    fig1, ax1 = plt.subplots(figsize=(8, 8))
    ax1.set_facecolor('black')
    fig1.patch.set_facecolor('black')
    
    for i in range(matriz_pesos.shape[0]):
        for j in range(matriz_pesos.shape[1]):
            if matriz_activaciones[i,j] > 0:
                ax1.add_patch(plt.Rectangle((i-0.5, j-0.5), 1, 1, 
                                         facecolor=matriz_pesos[i,j],
                                         edgecolor='black',
                                         linewidth=1))
    
    ax1.set_title('Mapa de Neuronas Activadas', color='white', pad=20)
    ax1.set_xlim(-1, matriz_pesos.shape[0])
    ax1.set_ylim(-1, matriz_pesos.shape[1])
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.grid(False)
    plt.show()
    
    # 2. Mapa de clasificación 3D
    fig2 = plt.figure(figsize=(12, 8))
    ax2 = fig2.add_subplot(111, projection='3d')
    x, y = np.meshgrid(range(matriz_activaciones.shape[0]), 
                       range(matriz_activaciones.shape[1]))
    z = matriz_activaciones.T
    
    dx = dy = 0.8
    norm = plt.Normalize(vmin=0, vmax=matriz_activaciones.max())
    cmap = cm.viridis
    
    ax2.set_zticks(np.arange(0, matriz_activaciones.max() + 1, 1))
    
    for i in range(matriz_activaciones.shape[0]):
        for j in range(matriz_activaciones.shape[1]):
            if matriz_activaciones[i,j] > 0:
                color = cmap(norm(matriz_activaciones[i,j]))
                ax2.bar3d(i-dx/2, j-dy/2, 0, dx, dy, z[j,i], 
                         color=color, shade=True, edgecolor='blue', linewidth=0.5)
    
    ax2.view_init(elev=25, azim=45)
    ax2.set_title('Mapa de Clasificación 3D', pad=20)
    ax2.set_box_aspect((1, 1, 0.7))
    
    mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
    cbar = plt.colorbar(mappable, ax=ax2, 
                       ticks=np.arange(0, matriz_activaciones.max() + 1, 1),
                       pad=0.1,
                       shrink=0.7,
                       aspect=20)
    cbar.set_label('Activaciones')
    plt.show()

def visualizar_mapa_distancias(matriz_pesos):
    """Visualiza el mapa de distancias entre neuronas adyacentes"""
    fig3, ax3 = plt.subplots(figsize=(8, 8))
    fig3.patch.set_facecolor('white')
    ax3.set_facecolor('white')
    
    lado_mapa = matriz_pesos.shape[0]
    distancias = np.zeros((lado_mapa, lado_mapa))
    
    for i in range(lado_mapa):
        for j in range(lado_mapa):
            vecinos = []
            for di, dj in [(0,1), (0,-1), (1,0), (-1,0)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < lado_mapa and 0 <= nj < lado_mapa:
                    dist = np.linalg.norm(matriz_pesos[i,j] - matriz_pesos[ni,nj])
                    vecinos.append(dist)
            distancias[i,j] = np.mean(vecinos) if vecinos else 0
    
    # Normalizar distancias
    distancias = (distancias - distancias.min()) / (distancias.max() - distancias.min())
    
    sns.heatmap(distancias, ax=ax3,
                cmap='viridis',
                cbar_kws={
                    'label': 'Distancia Normalizada',
                    'shrink': 0.7,
                    'aspect': 20,
                    'pad': 0.02
                },
                square=True,
                xticklabels=False,
                yticklabels=False)
    
    ax3.set_title('Mapa de Distancias', color='black', pad=20)
    ax3.collections[0].colorbar.ax.yaxis.set_tick_params(color='white')
    ax3.collections[0].colorbar.ax.tick_params(colors='white')
    plt.show()

# Execute classification and visualization
resultados, matriz_activaciones = clasificar_colores_prueba()

# Create DataFrame with results
df = pd.DataFrame(resultados)
print("\nTabla de resultados:")
print(df.to_string(index=False))

# Create metrics plots
plt.style.use('seaborn')
fig, axes = plt.subplots(3, 1, figsize=(12, 15))

# Plot metrics
axes[0].plot(range(len(df)), [r['Error'] for r in resultados], 
             marker='o', color='red', linestyle='-')
axes[0].set_title("Error de Cuantificación por Color de Prueba")
axes[0].set_xlabel("Color Index")
axes[0].set_ylabel("Error de Cuantificación")
axes[0].grid(True)

axes[1].plot(range(len(df)), [calcular_error_topologico([r['Color Original']/255], matriz_pesos_final, lado_mapa) 
                             for r in resultados], 
             marker='o', color='red', linestyle='-')
axes[1].set_title("Error Topológico por Color de Prueba")
axes[1].set_xlabel("Color Index")
axes[1].set_ylabel("Error Topológico")
axes[1].grid(True)

# Histogram of activations
color_names = ['Blanco', 'Rojo', 'Verde', 'Azul', 
               'Amarillo', 'Magenta', 'Cian', 'Negro']
axes[2].bar(color_names, [1]*len(color_names), color='red')
axes[2].set_title("Activación por Color de Prueba")
axes[2].set_xlabel("Color")
axes[2].set_ylabel("Activación")
axes[2].grid(True)
plt.setp(axes[2].xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

# Generate SOM visualizations
visualizar_mapas_som(matriz_pesos_final, matriz_activaciones)
visualizar_mapa_distancias(matriz_pesos_final)

In [ ]:
def test_nuevos_patrones(num_patrones=5):
    """
    Prueba el SOM con nuevos patrones de color y visualiza los resultados
    """
    # Generar nuevos colores aleatorios
    nuevos_patrones = np.random.randint(0, 256, size=(num_patrones, 3))
    nuevos_patrones_norm = nuevos_patrones / 255.0 if normalizar_datos else nuevos_patrones
    
    # Crear figura para visualización
    fig, axes = plt.subplots(num_patrones, 2, figsize=(8, 3*num_patrones))
    
    print("Clasificación de nuevos patrones:")
    for i in range(num_patrones):
        # Usar la versión normalizada para la clasificación
        patron = nuevos_patrones_norm[i]
        
        # Encontrar BMU para el nuevo patrón
        _, bmu_idx = calcular_bmu(patron, matriz_pesos_final)
        color_asignado = matriz_pesos_final[bmu_idx[0], bmu_idx[1]]
        
        # No es necesario convertir a CPU ya que usamos NumPy
        patron_rgb = nuevos_patrones[i]
        color_asignado_rgb = color_asignado
        
        # Asegurar que los colores estén en el rango correcto para visualización
        if normalizar_datos:
            color_asignado_rgb = color_asignado_rgb * 255
        
        print(f"\nPatrón {i+1}:")
        print(f"Color original RGB: {patron_rgb}")
        print(f"Posición en mapa: {bmu_idx}")
        print(f"Color asignado RGB: {color_asignado_rgb}")
        
        # Visualizar colores normalizados al rango [0,1]
        axes[i,0].add_patch(patches.Rectangle((0, 0), 1, 1, facecolor=patron_rgb/255))
        axes[i,0].set_title(f'Original {i+1}')
        axes[i,0].axis('off')
        
        axes[i,1].add_patch(patches.Rectangle((0, 0), 1, 1, facecolor=color_asignado_rgb/255))
        axes[i,1].set_title(f'SOM {i+1}')
        axes[i,1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Ejecutar prueba con 3 nuevos patrones
test_nuevos_patrones(3)